In [1]:
import transformers
import os
import re
from collections import Counter
from transformers import BertConfig, BertForMaskedLM, BertTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
import sys
import torch# Convert data to PyTorch tensors
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
import random
print(sys.version)
print(os.getcwd())

/home/qqi2/.conda/envs/transformers/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 15:12:24) [GCC 11.2.0]
/home/qqi2


In [5]:
#pip install torch

  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1

In [5]:
# initialize the pipeline
import torch
# here we choose whether to use gpu or cpu
if torch.cuda.is_available():
  device = "cuda:0"  # prefer gpu if available
  print('Torch on CUDA')
else:
  print('Torch on CPU')
  device = "cpu" # cpu if not

Torch on CUDA


In [3]:
#pip show torch

Name: torch
Version: 2.4.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/qqi2/.conda/envs/transformers/lib/python3.12/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, setuptools, sympy, triton, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#conda install -c conda-forge ipywidgets

In [ ]:
#pip install datasets

In [6]:
# Load your corpus
with open('data/wiki052024/data/he_wiki_cleaned.txt', 'r', encoding='utf-8') as f:
    corpus_file = [line.strip() for line in f if line.strip()]
print(type(corpus_file))
print(len(corpus_file))
print(corpus_file[0:10])

<class 'list'>
144581193
['political philosophy and movement anarchism is a political philosophy and movement that is against all forms of authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy typically including the state and capitalism', 'anarchism advocates for the replacement of the state with stateless societies and voluntary free associations', 'as a historically leftwing movement this reading of anarchism is placed on the farthest left of the political spectrum usually described as the libertarian wing of the socialist movement libertarian socialism', 'although traces of anarchist ideas are found all throughout history modern anarchism emerged from the enlightenment', 'during the latter half of the th and the first decades of the th century the anarchist movement flourished in most parts of the world and had a significant role in workers struggles for emancipation', 'various anarchist schools of thought formed during this period', 

In [14]:
len(corpus_file) * 0.0002

28916.2386

In [7]:
random.seed(42)

# Take a random sample of 2 sentences from the first 0.02% sentences
corpus_file_sample = random.sample(corpus_file, 28916)

print(type(corpus_file_sample))
print(len(corpus_file_sample))
print(corpus_file_sample[0:10])

<class 'list'>
28916
['müller had a strong start to the grass court season reaching the semifinals of the topshelf open as well as the quarterfinals at queens beating defending champion dimitrov for the second time in two meetings since the beginning of the year', 'the newspaper has had a circulation reaching', 'these two aircraft also proved underpowered and the type was abandoned with the last twin seaplane written off in', 'the diocese is a member of the conference of the latin bishops of the arab regions', 'because the early jurchens had no written records the story of hanpu was first transmitted orally', 'both clubs grew in stature as the s progressed leading to their first meeting of the s being billed as a meeting of two local clubs ardwick city just rising to the highest standard in the football world whilst newton heath are nearly in the front rank', 'after completing high school he entered syracuse university where he graduated in', 'some of these gaelic songs are about the l

In [8]:
# Specify the file path where you want to save the data
file_path = 'data/wiki052024/data/he_wiki_cleaned_sample.txt'

# Open the file in write mode and save the data
with open(file_path, 'w') as file:
    for line in corpus_file_sample:
        file.write(line + '\n')  # Write each line followed by a newline